# Circuit-level depolarizing noise: BP-OSD vs MWPM

In [1]:
from bp_osd import BP_OSD
from pymatching import Matching

import stim
import numpy as np

In [2]:
NUM_ROUNDS = 70 
NOISE_PROB = 0.01
DISTANCE = 3

circuit = stim.Circuit.generated(code_task="surface_code:rotated_memory_z", 
                                 distance=DISTANCE, 
                                 rounds=NUM_ROUNDS, 
                                 after_clifford_depolarization=NOISE_PROB)

In [3]:
sampler = circuit.compile_detector_sampler()
defects, log_flips = sampler.sample(shots=100_000, separate_observables=True)

In [4]:
bp_osd = BP_OSD.from_detector_error_model(circuit.detector_error_model())
bp_osd._matrices.check_matrix

<560x7723 sparse matrix of type '<class 'numpy.uint8'>'
	with 21468 stored elements in Compressed Sparse Column format>

In [5]:
mwpm = Matching(circuit.detector_error_model(decompose_errors=True))
predictions_mwpm = mwpm.decode_batch(defects)
print("prob logical error MWPM  ", 
      "| ave=", np.average(predictions_mwpm != log_flips), 
      "| std=", np.std(predictions_mwpm != log_flips)/np.sqrt(log_flips.shape[0]))

bp_osd = BP_OSD.from_detector_error_model(circuit.detector_error_model())
predictions_bposd = bp_osd.decode_batch(defects)
print("prob logical error BP-OSD", 
      "| ave=", np.average(predictions_bposd != log_flips), 
      "| std=", np.std(predictions_bposd != log_flips)/np.sqrt(log_flips.shape[0]))

for osd_order in [40]:
    bp_osd_cs = BP_OSD.from_detector_error_model(circuit.detector_error_model(), osd_method="osd_cs", osd_order=osd_order)
    predictions_bposd_cs = bp_osd_cs.decode_batch(defects)
    print(f"prob logical error BP-OSD(order={osd_order})", 
          "| ave=", np.average(predictions_bposd_cs != log_flips), 
          "| std=", np.std(predictions_bposd_cs != log_flips)/np.sqrt(log_flips.shape[0]))

prob logical error MWPM   | ave= 0.34564 | std= 0.0015039048852902898


100%|███████████████████████████████████| 100000/100000 [35:25<00:00, 47.06it/s]


prob logical error BP-OSD | ave= 0.37158 | std= 0.00152809784896125


100%|█████████████████████████████████| 100000/100000 [5:04:00<00:00,  5.48it/s]

prob logical error BP-OSD(order=40) | ave= 0.32444 | std= 0.0014804684609946942
